Create Test Set for Housing Data
Following the approach from "Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow"

This notebook demonstrates how to create a proper test set for the housing dataset
using stratified sampling based on income categories, as recommended in the book.
This ensures our test set is representative of the overall data distribution.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk


# Read the housing dataset
housing = pd.read_csv('/Users/marcelosilva/Desktop/Hands-on/data/raw/housing/housing.csv')

Creating Deterministic Dataset with Unique Identifiers

We opted to create a deterministic dataset to ensure reproducible results across different runs.
To achieve this, we'll use latitude and longitude coordinates to generate unique identifiers (UIDs)
for each housing record. This approach provides a consistent way to identify and track individual
properties while maintaining the spatial relationship inherent in the data.

The UIDs will be created using a hash function applied to the latitude and longitude coordinates
(rounded to 7 decimal places), ensuring each property has a unique identifier.

The processed dataset with UIDs will be saved to:
/Users/marcelosilva/Desktop/Hands-on/data/processed/housing

In [45]:
import hashlib

def make_uid(row, existing_uids):
    base_key = f"{row.latitude:.7f}_{row.longitude:.7f}"
    uid = hashlib.sha256(base_key.encode()).hexdigest()[:16]
    
    # If UID already exists, add disambiguation suffix
    counter = 1
    original_uid = uid
    while uid in existing_uids:
        uid = f"{original_uid}_{counter}"
        counter += 1
    
    existing_uids.add(uid)
    return uid

# Initialize set to track existing UIDs
existing_uids = set()

# Apply the function to create unique UIDs
housing["uid"] = housing.apply(lambda row: make_uid(row, existing_uids), axis=1)


In [ ]:
housing.to_csv("/Users/marcelosilva/Desktop/Hands-on/data/processed/housing/housing_with_uid.csv", index=False)


Feature Balance Analysis

Before proceeding with the test set creation, we need to evaluate the distribution and balance of our features.
This analysis will help us understand:

- **Feature Distributions**: How each feature is distributed across the dataset
- **Class Balance**: For categorical features, how balanced the classes are
- **Outlier Detection**: Identify potential outliers that might affect our model
- **Correlation Analysis**: Understand relationships between features

This evaluation will inform our test set creation strategy and help ensure that our train/test split
 maintains representative distributions of all features.

In [47]:
# Get all features from the dataframe
features = housing.columns.tolist()
print(f"Total features in dataset: {len(features)}")
print(f"Features: {features}")

# Separate numerical and categorical features
numerical_features = housing.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = housing.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\nNumerical features ({len(numerical_features)}): {numerical_features}")
print(f"Categorical features ({len(categorical_features)}): {categorical_features}")

# Basic statistics for numerical features
print("\n" + "="*50)
print("NUMERICAL FEATURES ANALYSIS")
print("="*50)
print(housing[numerical_features].describe())

# Distribution analysis for categorical features
print("\n" + "="*50)
print("CATEGORICAL FEATURES ANALYSIS")
print("="*50)
for feature in categorical_features:
    print(f"\n{feature}:")
    value_counts = housing[feature].value_counts()
    print(f"Unique values: {len(value_counts)}")
    print(f"Most common: {value_counts.head(3).to_dict()}")
    print(f"Missing values: {housing[feature].isnull().sum()}")

# Check for outliers in numerical features using IQR method
print("\n" + "="*50)
print("OUTLIER DETECTION (IQR Method)")
print("="*50)
for feature in numerical_features:
    if feature != 'uid':  # Skip UID as it's not a meaningful feature for outlier detection
        Q1 = housing[feature].quantile(0.25)
        Q3 = housing[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = housing[(housing[feature] < lower_bound) | (housing[feature] > upper_bound)]
        print(f"{feature}: {len(outliers)} outliers ({len(outliers)/len(housing)*100:.2f}%)")

# Correlation analysis for numerical features
print("\n" + "="*50)
print("CORRELATION ANALYSIS")
print("="*50)
correlation_matrix = housing[numerical_features].corr()
print("Correlation matrix shape:", correlation_matrix.shape)
print("\nTop correlations (absolute value > 0.5):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.5:
            print(f"{correlation_matrix.columns[i]} - {correlation_matrix.columns[j]}: {corr_value:.3f}")

# Missing values analysis
print("\n" + "="*50)
print("MISSING VALUES ANALYSIS")
print("="*50)
missing_values = housing.isnull().sum()
missing_percentage = (missing_values / len(housing)) * 100
missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})
print(missing_df[missing_df['Missing Values'] > 0])


Total features in dataset: 11
Features: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'uid']

Numerical features (9): ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']
Categorical features (2): ['ocean_proximity', 'uid']

NUMERICAL FEATURES ANALYSIS
          longitude      latitude  housing_median_age   total_rooms  \
count  20640.000000  20640.000000        20640.000000  20640.000000   
mean    -119.569704     35.631861           28.639486   2635.763081   
std        2.003532      2.135952           12.585558   2181.615252   
min     -124.350000     32.540000            1.000000      2.000000   
25%     -121.800000     33.930000           18.000000   1447.750000   
50%     -118.490000     34.260000           29.000000   2127.000000   
75%     -118.010000     37.710000 

In [ ]:
df_com_uid = pd.read_csv('/Users/marcelosilva/Desktop/Hands-on/data/processed/housing/housing_with_uid.csv')

In [49]:
# Check if all UID values are unique in df_com_uid
print("="*50)
print("UID UNIQUENESS CHECK")
print("="*50)

total_uid_count = len(df_com_uid['uid'])
unique_uid_count = df_com_uid['uid'].nunique()

print(f"Total UID count: {total_uid_count}")
print(f"Unique UID count: {unique_uid_count}")
print(f"Are all UIDs unique? {total_uid_count == unique_uid_count}")

if total_uid_count != unique_uid_count:
    print(f"Number of duplicate UIDs: {total_uid_count - unique_uid_count}")
    # Show duplicate UIDs if any exist
    duplicates = df_com_uid[df_com_uid['uid'].duplicated(keep=False)]
    print(f"Duplicate UIDs found: {len(duplicates)} rows")
    print("Sample of duplicate UIDs:")
    print(duplicates[['uid']].head(10))
else:
    print("✓ All UIDs are unique!")



UID UNIQUENESS CHECK
Total UID count: 20640
Unique UID count: 20640
Are all UIDs unique? True
✓ All UIDs are unique!


In [50]:
# Investigate why UIDs are not unique when they should be based on latitude and longitude
print("="*50)
print("INVESTIGATING UID DUPLICATES")
print("="*50)

# Check for duplicate UIDs
duplicate_uids = df_com_uid[df_com_uid['uid'].duplicated(keep=False)].sort_values('uid')

if len(duplicate_uids) > 0:
    print(f"Found {len(duplicate_uids)} rows with duplicate UIDs")
    print(f"Number of unique duplicate UID values: {duplicate_uids['uid'].nunique()}")
    
    # Show first few duplicate UIDs with their lat/lon values
    print("\nFirst few duplicate UIDs with their coordinates:")
    for uid in duplicate_uids['uid'].unique()[:5]:
        uid_rows = df_com_uid[df_com_uid['uid'] == uid]
        print(f"\nUID: {uid}")
        print(uid_rows[['uid', 'latitude', 'longitude']].to_string(index=False))
        
        # Check if lat/lon are actually the same
        lat_unique = uid_rows['latitude'].nunique()
        lon_unique = uid_rows['longitude'].nunique()
        print(f"  Unique latitude values: {lat_unique}")
        print(f"  Unique longitude values: {lon_unique}")
        
        if lat_unique == 1 and lon_unique == 1:
            print("  → Same coordinates, UID generation is correct")
        else:
            print("  → Different coordinates with same UID - this is the problem!")
            
    # Check if there are any rows with same lat/lon but different UIDs
    print("\n" + "="*30)
    print("CHECKING FOR SAME COORDINATES WITH DIFFERENT UIDS")
    print("="*30)
    
    # Group by lat/lon and check for multiple UIDs
    coord_groups = df_com_uid.groupby(['latitude', 'longitude'])
    same_coord_diff_uid = []
    
    for (lat, lon), group in coord_groups:
        if group['uid'].nunique() > 1:
            same_coord_diff_uid.append((lat, lon, group))
    
    if same_coord_diff_uid:
        print(f"Found {len(same_coord_diff_uid)} coordinate pairs with multiple UIDs!")
        for lat, lon, group in same_coord_diff_uid[:3]:
            print(f"\nCoordinates: ({lat}, {lon})")
            print(f"UIDs: {group['uid'].unique()}")
    else:
        print("✓ No coordinate pairs with multiple UIDs found")
        
else:
    print("✓ No duplicate UIDs found - this is unexpected given the previous check!")

# Additional investigation: Check UID generation logic
print("\n" + "="*30)
print("UID GENERATION INVESTIGATION")
print("="*30)

# Sample a few rows to see how UID was generated
sample_rows = df_com_uid.head(10)
print("Sample UID generation:")
for _, row in sample_rows.iterrows():
    expected_uid = f"{row['latitude']:.6f}_{row['longitude']:.6f}"
    actual_uid = row['uid']
    print(f"Lat: {row['latitude']:.6f}, Lon: {row['longitude']:.6f}")
    print(f"  Expected UID: {expected_uid}")
    print(f"  Actual UID:   {actual_uid}")
    print(f"  Match: {expected_uid == actual_uid}")
    print()


INVESTIGATING UID DUPLICATES
✓ No duplicate UIDs found - this is unexpected given the previous check!

UID GENERATION INVESTIGATION
Sample UID generation:
Lat: 37.880000, Lon: -122.230000
  Expected UID: 37.880000_-122.230000
  Actual UID:   89880b50888801da
  Match: False

Lat: 37.860000, Lon: -122.220000
  Expected UID: 37.860000_-122.220000
  Actual UID:   afc03886ba0e449b
  Match: False

Lat: 37.850000, Lon: -122.240000
  Expected UID: 37.850000_-122.240000
  Actual UID:   357450aa81fd6e6f
  Match: False

Lat: 37.850000, Lon: -122.250000
  Expected UID: 37.850000_-122.250000
  Actual UID:   bd9be4855ebbd493
  Match: False

Lat: 37.850000, Lon: -122.250000
  Expected UID: 37.850000_-122.250000
  Actual UID:   bd9be4855ebbd493_1
  Match: False

Lat: 37.850000, Lon: -122.250000
  Expected UID: 37.850000_-122.250000
  Actual UID:   bd9be4855ebbd493_2
  Match: False

Lat: 37.840000, Lon: -122.250000
  Expected UID: 37.840000_-122.250000
  Actual UID:   fce15d48192cb9ac
  Match: False

L